[![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/camenduru/stable-diffusion-webui-sagemaker/blob/main/nightly/chillout_mix_webui_sagemaker.ipynb)

In [ ]:
# パッケージのインストール → カーネルを再起動する
!mkdir /home/studio-lab-user/content
%cd /home/studio-lab-user/content

%env TF_CPP_MIN_LOG_LEVEL=1

%conda install -q -y aria2
%conda install -q -y libglib2.0-0
# %conda install -q -y -c conda-forge libglib
%conda install -q -y glib
%pip install -q opencv-python-headless huggingface-hub
!pip install -q torch>=2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.18 triton==2.0.0 -U

!git clone -b v2.2 https://github.com/camenduru/stable-diffusion-webui

# embed
!git clone https://huggingface.co/embed/negative /home/studio-lab-user/content/stable-diffusion-webui/embeddings/negative

# 時刻
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O /home/studio-lab-user/content/stable-diffusion-webui/scripts/run_n_times.py

In [ ]:
# BRAV6 - Model
!rm -rf  /home/studio-lab-user/content/stable-diffusion-webui/models/Stable-diffusion/*
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/okozk/myBRAV6_down/resolve/main/beautifulRealistic_v60.safetensors -d /home/studio-lab-user/content/stable-diffusion-webui/models/Stable-diffusion -o beautifulRealistic_v60.safetensors
# chilled_remix_v2 - Model
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/sazyou-roukaku/chilled_remix/resolve/main/chilled_remix_v2.safetensors -d /home/studio-lab-user/content/stable-diffusion-webui/models/Stable-diffusion -o chilled_remix_v2.safetensors

# VAE
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d /home/studio-lab-user/content/stable-diffusion-webui/models/VAE -o vae-ft-mse-840000-ema-pruned.vae.pt

!ls -la /home/studio-lab-user/content/stable-diffusion-webui/models/Stable-diffusion
!ls -la /home/studio-lab-user/content/stable-diffusion-webui/models/VAE
!rm -rf /home/studio-lab-user/content/=2.0.0+cu118

In [ ]:
!rm -rf /home/studio-lab-user/content/stable-diffusion-webui/embeddings/*
!rm -rf /home/studio-lab-user/content/stable-diffusion-webui/models/Lora/*

# embedings
!git clone https://huggingface.co/okozk/embedings /home/studio-lab-user/content/stable-diffusion-webui/embeddings
!rm -rf /home/studio-lab-user/content/stable-diffusion-webui/embeddings/README.md
!rm -rf /home/studio-lab-user/content/stable-diffusion-webui/embeddings/.gitattributes
!rm -rf /home/studio-lab-user/content/stable-diffusion-webui/embeddings/.git
!rm -rf /home/studio-lab-user/content/stable-diffusion-webui/embeddings/.ipynb_checkpoints

# Lora
!git clone https://huggingface.co/okozk/Lora /home/studio-lab-user/content/stable-diffusion-webui/models/Lora/
!rm -rf /home/studio-lab-user/content/stable-diffusion-webui/models/Lora/README.md
!rm -rf /home/studio-lab-user/content/stable-diffusion-webui/models/Lora/.gitattributes
!rm -rf /home/studio-lab-user/content/stable-diffusion-webui/models/Lora/.git
!rm -rf /home/studio-lab-user/content/stable-diffusion-webui/models/Lora/.ipynb_checkpoints

!ls -la /home/studio-lab-user/content/stable-diffusion-webui/embeddings
!ls -la /home/studio-lab-user/content/stable-diffusion-webui/models/Lora

In [ ]:
# 拡張機能の追加
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser /home/studio-lab-user/content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git clone https://github.com/Mikubill/sd-webui-controlnet /home/studio-lab-user/content/stable-diffusion-webui/extensions/sd-webui-controlnet
!git clone https://github.com/jexom/sd-webui-depth-lib /home/studio-lab-user/content/stable-diffusion-webui/extensions/sd-webui-depth-lib
!git clone https://github.com/richrobber2/canvas-zoom /home/studio-lab-user/content/stable-diffusion-webui/extensions/canvas-zoom
!git clone https://github.com/Zyin055/Config-Presets /home/studio-lab-user/content/stable-diffusion-webui/extensions/Config-Presets
!git clone https://github.com/alemelis/sd-webui-ar /home/studio-lab-user/content/stable-diffusion-webui/extensions/sd-webui-ar
!git clone https://github.com/aka7774/sd_katanuki /home/studio-lab-user/content/stable-diffusion-webui/extensions/sd_katanuki
!git clone https://github.com/camenduru/sd-civitai-browser /home/studio-lab-user/content/stable-diffusion-webui/extensions/sd-civitai-browser
!git clone https://github.com/Bing-su/adetailer /home/studio-lab-user/content/stable-diffusion-webui/extensions/adetailer

# リロード
%cd /home/studio-lab-user/content/stable-diffusion-webui
!git reset --hard
!git -C /home/studio-lab-user/content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai reset --hard

In [ ]:
# ControlNetのモデルの追加
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d /home/studio-lab-user/content/stable-diffusion-webui/models/ControlNet -o control_v11f1p_sd15_depth_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d /home/studio-lab-user/content/stable-diffusion-webui/models/ControlNet -o control_v11p_sd15_inpaint_fp16.safetensors
!ls /home/studio-lab-user/content/stable-diffusion-webui/models/ControlNet

In [ ]:
# depth-lib の画像をインストールする ⭐ maps.zip を用意しておく
!unzip -d /home/studio-lab-user/content/stable-diffusion-webui/extensions/sd-webui-depth-lib/maps/ /home/studio-lab-user/maps.zip > /dev/null
!rm -rf /home/studio-lab-user/maps.zip
!rm -rf /home/studio-lab-user/content/stable-diffusion-webui/extensions/sd-webui-depth-lib/maps/__MACOSX
!ls /home/studio-lab-user/content/stable-diffusion-webui/extensions/sd-webui-depth-lib/maps

In [ ]:
******************************************************************************
Optional
******************************************************************************

In [ ]:
# sd_katanuki のフォルダを用意 ⭐ train-images.zip を用意しておく
!mkdir /home/studio-lab-user/sd_katanuki_in
!mkdir /home/studio-lab-user/sd_katanuki_out
!unzip -d /home/studio-lab-user/sd_katanuki_in /home/studio-lab-user/train-images.zip > /dev/null
!rm -rf /home/studio-lab-user/sd_katanuki_in/__MACOSX
!ls /home/studio-lab-user/sd_katanuki_in # | wc -l

In [ ]:
# sd_katanuki の結果をZipにする
%cd /home/studio-lab-user
!tar -czf /home/studio-lab-user/sd_katanuki_out.tar.gz sd_katanuki_out
!rm -rf /home/studio-lab-user/sd_katanuki_in
!rm -rf /home/studio-lab-user/sd_katanuki_out
!rm -rf /home/studio-lab-user/train-images.zip

In [ ]:
# 画像生成 の結果をZipにする
%cd /home/studio-lab-user/content/stable-diffusion-webui

import datetime
today = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
!tar -czf /home/studio-lab-user/outputs_{today}.tar.gz outputs

%cd /home/studio-lab-user
!rm -rf /home/studio-lab-user/content/stable-diffusion-webui/outputs/txt2img-images/*
!rm -rf /home/studio-lab-user/content/stable-diffusion-webui/outputs/txt2img-grids/*
!rm -rf /home/studio-lab-user/content/stable-diffusion-webui/outputs/img2img-images/*
!rm -rf /home/studio-lab-user/content/stable-diffusion-webui/outputs/img2img-grids/*
!rm -rf /home/studio-lab-user/content/stable-diffusion-webui/outputs/extras-images/*

In [ ]:
# キャッシュの削除
%cd ~/
!pwd
!rm -rf .cache
!ls -la
# %pip install

In [ ]:
******************************************************************************

In [ ]:
# ⭐ 実行 ##############################################
%cd /home/studio-lab-user/content/stable-diffusion-webui

!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /home/studio-lab-user/content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /home/studio-lab-user/content/stable-diffusion-webui/launch.py
!sed -i -e 's/\"sd_model_checkpoint\"\,/\"sd_model_checkpoint\,sd_vae\,CLIP_stop_at_last_layers\"\,/g' /home/studio-lab-user/content/stable-diffusion-webui/modules/shared.py

!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --ngrok "2SOs8DsoZvkiDNBFjisetiIZJqu_7XBuj79rA4yeX3Bxas3Dc"


100%|███████████████████████████████████████████| 30/30 [00:45<00:00,  1.51s/it]
mediapipe: 1 detected.
100%|███████████████████████████████████████████| 30/30 [00:07<00:00,  4.04it/s]
	Tile 1/12
	Tile 2/12
	Tile 3/12
	Tile 4/12
	Tile 5/12
	Tile 6/12
	Tile 7/12
	Tile 8/12
	Tile 9/12
	Tile 10/12
	Tile 11/12
	Tile 12/12
100%|███████████████████████████████████████████| 30/30 [00:45<00:00,  1.50s/it]
[-] ADetailer: nothing detected on image 21 with 1st settings.
100%|███████████████████████████████████████████| 30/30 [00:07<00:00,  4.07it/s]
	Tile 1/12
	Tile 2/12
	Tile 3/12
	Tile 4/12
	Tile 5/12
	Tile 6/12
	Tile 7/12
	Tile 8/12
	Tile 9/12
	Tile 10/12
	Tile 11/12
	Tile 12/12
100%|███████████████████████████████████████████| 30/30 [00:44<00:00,  1.50s/it]
mediapipe: 1 detected.
 69%|██████████████████████████████▍             | 9/13 [00:02<00:00,  4.07it/s]